# HOTEL HR PEOPLE ANALYTICS

### Hotel Analysis from a People Analytics perspective.
<p>This project will try to emulate HR analytics to continue practicing my analytical skills. Also, with this project, I want to <u>strengthen my knowledge</u> in <b><i>MySql</b></i> <b><i>and SQL</i></b>. I will use <b><i>Python</i></b> and <b><i>Power BI</i></b> for the data analysis.</p>
<p>I will start with doing a little of data engineering to design the databases I am going to work with. I will use <b><i>Figma</i></b> to draft the databases and the relationships between each other. Then I will fill the databases with the data, and later I will start the data analysis.</p>

### 1. Import libraries

In [1]:
# Libraries to manipulate the data
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string

# Library to deploy charts with the data
import seaborn as sns
import matplotlib.pyplot as plt

# Statmodels for predictions
import statsmodels.api as sm
import statsmodels.formula.api as smf

# This is to ignore warnings.
import warnings
warnings.filterwarnings('ignore')

### 2. Working with our databases

<p>Previously, I created the databases I was going to work with. Because part of the data was difficult to create randomly, I downloaded the databases into an <i>.xlsx</i> file and worked with it.

<p>I called the file <b><i>hotel_hranalytics.xlsx</i></b>, so now it's time to start working with it.</p>

In [5]:
# Let's load our databases
df_rawemp = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Employees')

df_rawemp.head()

,emp_id,Name,Surname,Birthday,Age,Gender,on_license,hotel_id
0,3272,James,Smith,1957-08-09,67,M,0,FUESSP
1,3073,John,Johnson,1981-11-19,42,M,0,FUESSP
2,6627,Robert,Williams,1983-10-15,41,M,0,FUESSP
3,420,Michael,Brown,1976-04-05,48,M,0,FUESSP
4,4856,William,Jones,1968-11-20,55,M,0,FUESSP


In [6]:
df_rawht = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Hotels')

df_rawht.head()

,hotel_id,Name,Location,Opening,Stars,Budget
0,FUESSP,Sandy Shores Park,28 03 18.9N-14 19 21.4W,2001-03-05,4,350000000
1,TFNOBH,Ocean Breeze Haven,28 05 56.5N-16 44 54.6W,1998-10-05,5,550000000
2,ACECWR,Coral Wave Resort,28 51 25.9N-13 47 48.7 W,2000-05-05,5,480000000


In [7]:
df_rawhtcomp = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Hotel_Composition')

df_rawhtcomp.head()

,hc_id,Department,Active_employees,Emp_with_license,Total_employees
0,FUESSP,Reception_Reservations,11,1,12
1,FUESSP,Floors_Laundry,35,3,38
2,FUESSP,Kitchen,35,3,38
3,FUESSP,Bar_Restaurant,31,7,38
4,FUESSP,Animation,11,1,12


In [8]:
df_rawempwages = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Employees_wages')

df_rawempwages.head()

,emp_wag_id,Price_$_Hour,Hours_worked,Work_overtime,Ovh$_75%,Gross_pay,Deductions_3%,Total_Payment,emp_id,hotel_id,Payment_date
0,3272REFUESSP,14,129,4,10.50,1848.00,55.4400,1792.5600,3272,FUESSP,2024-01-29
1,3073REFUESSP,14,143,3,10.50,2033.50,61.0050,1972.4950,3073,FUESSP,2024-01-29
2,6627REFUESSP,18,135,4,13.50,2484.00,74.5200,2409.4800,6627,FUESSP,2024-01-29
3,420REFUESSP,19,121,11,14.25,2455.75,73.6725,2382.0775,420,FUESSP,2024-01-29
4,4856REFUESSP,14,132,7,10.50,1921.50,57.6450,1863.8550,4856,FUESSP,2024-01-29


In [9]:
df_rawworkforce = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Workforce_Composition')

df_rawworkforce.head()

,wkc_id,Department,Position,years_at_position,Entry_date,years_working,Staff,emp_id,hotel_id,Unnamed: 9,Unnamed: 10
0,3272FUESSP,Reception_Reservations,Staff,1,2023-09-26,1,0,3272,FUESSP,NaN,NaN
1,3073FUESSP,Reception_Reservations,Staff,1,2023-04-29,1,0,3073,FUESSP,NaN,NaN
2,6627FUESSP,Reception_Reservations,3rd_Command,4,2014-01-17,10,7,6627,FUESSP,NaN,NaN
3,420FUESSP,Reception_Reservations,3rd_Command,3,2012-10-25,12,7,420,FUESSP,NaN,NaN
4,4856FUESSP,Reception_Reservations,Staff,1,2023-06-18,1,0,4856,FUESSP,NaN,NaN
